In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_numopt
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [8]:
class Net(nn.Module):
    def __init__(self, input_size, device="cpu"):
        super().__init__()
        self.f1 = nn.Linear(input_size, 10, device=device)
        self.f2 = nn.Linear(10, 20, device=device)
        self.f3 = nn.Linear(20, 20, device=device)
        self.f4 = nn.Linear(20, 10, device=device)
        self.f5 = nn.Linear(10, 1, device=device)

        self.activation = nn.ReLU()
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.f1(x))
        x = self.activation(self.f2(x))
        x = self.activation(self.f3(x))
        x = self.activation(self.f4(x))
        x = self.f5(x)

        return x

In [9]:
# X, y = load_diabetes(return_X_y=True, scaled=False)
# X, y = make_regression(n_samples=1000, n_features=100)
X, y = make_friedman1(n_samples=1000, noise=1e-2)

X_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)
X = torch.Tensor(X).to(device)

y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y.reshape((-1, 1)))
y = torch.Tensor(y).to(device)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

torch_data = TensorDataset(X_train, y_train)
data_loader = DataLoader(torch_data, batch_size=1000)

torch.Size([800, 10]) torch.Size([800, 1])
torch.Size([200, 10]) torch.Size([200, 1])


In [10]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = torch_numopt.LM(
    params=model.parameters(),
    model=model,
    lr=1,
    mu=0.001,
    mu_dec=0.1,
    use_diagonal=False,
    c1=1e-4,
    tau=0.1,
    line_search_method="backtrack",
    line_search_cond="armijo",
)

times = []

all_loss = {}
for epoch in range(100):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].cpu().detach().numpy().item()))

epoch:  0, loss: 0.4964022636413574
epoch:  1, loss: 0.48567652702331543
epoch:  2, loss: 0.4739667475223541
epoch:  3, loss: 0.46143224835395813
epoch:  4, loss: 0.44932645559310913
epoch:  5, loss: 0.4369351863861084
epoch:  6, loss: 0.42492401599884033
epoch:  7, loss: 0.4127388596534729
epoch:  8, loss: 0.4008500874042511
epoch:  9, loss: 0.3888336718082428
epoch:  10, loss: 0.37683746218681335
epoch:  11, loss: 0.3654038906097412
epoch:  12, loss: 0.35359007120132446
epoch:  13, loss: 0.3420681357383728
epoch:  14, loss: 0.33061736822128296
epoch:  15, loss: 0.31905990839004517
epoch:  16, loss: 0.3076438009738922
epoch:  17, loss: 0.2958626449108124
epoch:  18, loss: 0.284471720457077
epoch:  19, loss: 0.2734847664833069
epoch:  20, loss: 0.2625884711742401
epoch:  21, loss: 0.2518167197704315
epoch:  22, loss: 0.24116618931293488
epoch:  23, loss: 0.2306242287158966
epoch:  24, loss: 0.22034727036952972
epoch:  25, loss: 0.21011443436145782
epoch:  26, loss: 0.200198233127594
ep

In [11]:
pred_train = model.forward(X_train).cpu().detach()
pred_test = model.forward(X_test).cpu().detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train.cpu())}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test.cpu())}")

Train metrics: R2 = 0.9750874518780936
Test metrics:  R2 = 0.9620878160190012


In [12]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = torch_numopt.LM(
    params=model.parameters(),
    model=model,
    lr=1,
    mu=0.001,
    mu_dec=0.1,
    use_diagonal=True,
    c1=1e-4,
    tau=0.1,
    line_search_method="backtrack",
    line_search_cond="armijo",
)

all_loss = {}
for epoch in range(100):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].cpu().detach().numpy().item()))

epoch:  0, loss: 0.100897878408432
epoch:  1, loss: 0.09353119879961014
epoch:  2, loss: 0.08680525422096252
epoch:  3, loss: 0.0791642814874649
epoch:  4

/home/eugeniolr/Documents/invAI/torch_numopt/src/torch_numopt/utils.py:33: UserWarning: torch.linalg.svd: During SVD computation with the selected cusolver driver, batches 0 failed to converge. A more accurate method will be used to compute the SVD as a fallback. Check doc at https://pytorch.org/docs/stable/generated/torch.linalg.svd.html (Triggered internally at /pytorch/aten/src/ATen/native/cuda/linalg/BatchLinearAlgebraLib.cpp:690.)
  U, S, Vt = torch.linalg.svd(mat)


, loss: 0.07399028539657593
epoch:  5, loss: 0.06732853502035141
epoch:  6, loss: 0.06676404923200607
epoch:  7, loss: 0.06487033516168594
epoch:  8, loss: 0.05817955732345581
epoch:  9, loss: 0.053729791194200516
epoch:  10, loss: 0.04843919351696968
epoch:  11, loss: 0.04360366612672806
epoch:  12, loss: 0.038669612258672714
epoch:  13, loss: 0.034483425319194794
epoch:  14, loss: 0.030952658504247665
epoch:  15, loss: 0.02741706557571888
epoch:  16, loss: 0.024480389431118965
epoch:  17, loss: 0.021885279566049576
epoch:  18, loss: 0.019750535488128662
epoch:  19, loss: 0.017851192504167557
epoch:  20, loss: 0.016019117087125778
epoch:  21, loss: 0.014269902370870113
epoch:  22, loss: 0.01269958820194006
epoch:  23, loss: 0.011302256025373936
epoch:  24, loss: 0.010109484195709229
epoch:  25, loss: 0.00909316074103117
epoch:  26, loss: 0.008269531652331352
epoch:  27, loss: 0.007492590229958296
epoch:  28, loss: 0.006822254508733749
epoch:  29, loss: 0.006216709036380053
epoch:  30,

In [14]:
pred_train = model.forward(X_train).cpu().detach()
pred_test = model.forward(X_test).cpu().detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train.cpu())}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test.cpu())}")

Train metrics: R2 = 0.9895769623159759
Test metrics:  R2 = 0.9821004910719348
